# Exploration_SSAC 06 멋진 작사가 만들기

* Keyword :: Sequence-to-Sequence , corpus , Tokenization , Source Sentence , Target Sentence

##### Sequence Data (시계열 데이터)

데이터를 순서대로 하나씩 나열하여 나타낸 데이터 구조.  
Sequence 간의 특정 관계나 패턴이 존재하지 않아도 특정 위치를 지정할 수 있는, 지정된 데이터를 의미한다.  

##### Sequence to Sequence :: Seq2Seq  

말 그대로, 시계열 데이터를 다른 시계열 데이터로 변환하는 구조를 가진 모델  
RNN (LSTM) Network 기반 Encoder + Decoder 파트로 구성된 모형  

##### Corpus (말뭉치)   
우리가 사용하는 텍스트 표본  
텍스트의 단락, 페이지 등 텍스트가 담긴 뭉치

##### Tokenization (토큰화)   
문서를 토큰(token)이라 불리는 단위로 나누는 작업  
**토큰의 기준** : 보통 단어(word)를 기준, 이외에도 문자(철자) 또는 구(phase), 문장(sentence), 단락(paragraph) 등을 기준으로 할 수 있음

##### 자연어 처리 데이터 정제 / 전처리  
raw_corpus --> corpus --> token --> 데이터 정제 --> 데이터 정규화 --> Dataset 객체화

##### Project Preview

* Dataset : kaggle 에 업로드된 **Song Lyrics**   
    49개의 노래 가사가 담긴 TXT 파일

##### Project Process  

1) 데이터 다운로드  
2) 데이터 로드  
3) 데이터 전처리 / 정제  
4) 평가 데이터 셋 분리  
5) 학습 모델 생성  

##### 필요한 모듈 import

In [1]:
import glob # 데이터 읽어올 때 유용한 라이브러리
import os 
import re                  # 정규표현식을 위한 Regex 지원 모듈 (문장 데이터를 정돈하기 위해) 
import numpy as np         # 변환된 문장 데이터(행렬)을 편하게 처리하기 위해
import tensorflow as tf
from sklearn.model_selection import train_test_split

## 1) 데이터 다운로드

* 직접 kaggle 페이지 https://www.kaggle.com/paultimothymooney/poetry/data 에서 다운로드 가능  
* 터미널에서 wget으로 다운로드 및 압축풀기 가능

## 2) 데이터 로드

In [2]:
txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*' # 데이터 로드 경로 설정

txt_list = glob.glob(txt_file_path)

raw_corpus = [] # 데이터 전처리가 들어가기 전 raw data 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

# 현재까지 raw_corpus 는 한 줄 (line) 단위로 끊겨 있음
print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Look, I was gonna go easy on you and not to hurt your feelings', "But I'm only going to get this one chance", "Something's wrong, I can feel it (Six minutes, Slim Shady, you're on)"]


## 3) 데이터 전처리 / 정제

데이터를 학습시키기 효율적이게 불필요한 요소는 제거하는 전처리 단계를 거치자.  
추가적으로, 지나치게 긴 문장은 과도한 padding을 가지게 함으로서 이 또한 비효율적 데이터가 될 수 있으므로 제거.  

토큰화 했을 때, 토큰의 개수가 15개를 넘어가는 문장을 학습데이터에서 제외하기 

* 정규표현식을 이용한 corpus(말뭉치) 생성  
* tf.keras.preprocessing.text.Tokenizer 이용하여 corpus를 텐서로 변환  
* tf.data.Dataset.from_tensor_slices() 이용하여 corpus 텐서를 tf.data.Dataset 객체로 변환

##### 예제에서 나온 부분이고, 부합하지 않을 수도 있지만 실행

In [4]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx > 9: break   # 일단 문장 10개만 확인해 볼 겁니다.
        
    print(sentence)

Look, I was gonna go easy on you and not to hurt your feelings
But I'm only going to get this one chance
Something's wrong, I can feel it (Six minutes, Slim Shady, you're on)
Just a feeling I've got, like something's about to happen, but I don't know what
If that means, what I think it means, we're in trouble, big trouble,
And if he is as bananas as you say, I'm not taking any chances
You were just what the doctor ordered I'm beginning to feel like a Rap God, Rap God
All my people from the front to the back nod, back nod
Now who thinks their arms are long enough to slap box, slap box?
They said I rap like a robot, so call me Rapbot But for me to rap like a computer must be in my genes


* raw_corpus 말뭉치가 한 단계 정리된 문장을 가지고 한 번 더 '단어' 별로 나누는 토큰화를 진행

In [5]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 이 문장이 어떻게 필터링되는지 확인해 보세요.

<start> this is sample sentence . <end>


In [6]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> look , i was gonna go easy on you and not to hurt your feelings <end>',
 '<start> but i m only going to get this one chance <end>',
 '<start> something s wrong , i can feel it six minutes , slim shady , you re on <end>',
 '<start> just a feeling i ve got , like something s about to happen , but i don t know what <end>',
 '<start> if that means , what i think it means , we re in trouble , big trouble , <end>',
 '<start> and if he is as bananas as you say , i m not taking any chances <end>',
 '<start> you were just what the doctor ordered i m beginning to feel like a rap god , rap god <end>',
 '<start> all my people from the front to the back nod , back nod <end>',
 '<start> now who thinks their arms are long enough to slap box , slap box ? <end>',
 '<start> they said i rap like a robot , so call me rapbot but for me to rap like a computer must be in my genes <end>']

##### 정제된 데이터를 활용하여 토큰화 진행  

토큰화하고, 단어 사전을 만들어주며, 데이터를 숫자로 변환까지 한 번에 진행   
:: 벡터화 -> 숫자로 변환된 데이터를 텐서(tensor)

In [7]:
corpus

['<start> look , i was gonna go easy on you and not to hurt your feelings <end>',
 '<start> but i m only going to get this one chance <end>',
 '<start> something s wrong , i can feel it six minutes , slim shady , you re on <end>',
 '<start> just a feeling i ve got , like something s about to happen , but i don t know what <end>',
 '<start> if that means , what i think it means , we re in trouble , big trouble , <end>',
 '<start> and if he is as bananas as you say , i m not taking any chances <end>',
 '<start> you were just what the doctor ordered i m beginning to feel like a rap god , rap god <end>',
 '<start> all my people from the front to the back nod , back nod <end>',
 '<start> now who thinks their arms are long enough to slap box , slap box ? <end>',
 '<start> they said i rap like a robot , so call me rapbot but for me to rap like a computer must be in my genes <end>',
 '<start> i got a laptop in my back pocket <end>',
 '<start> my pen ll go off when i half cock it <end>',
 '<sta

In [8]:
len(corpus[0]) # 각 corpus 내의 '글자/알파벳' 수

77

In [9]:
len(corpus) # 전체 리스트에서 corpus 데이터의 개수

175749

In [10]:
def tokenize(corpus):
    
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   4    5   57 ...   21 1079    3]
 [   2   36    5 ...    0    0    0]
 [   4    5   32 ...   54   18    3]
 ...
 [   2    7  224 ...    0    0    0]
 [  19  144    4 ...   19  881    3]
 [   2    8 3426 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fc44ff23f50>


In [11]:
corpus[0]

'<start> look , i was gonna go easy on you and not to hurt your feelings <end>'

In [12]:
tokenizer

In [13]:
tokenizer.index_word.items()

dict_items([(1, '<unk>'), (2, '<start>'), (3, '<end>'), (4, ','), (5, 'i'), (6, 'the'), (7, 'you'), (8, 'and'), (9, 'a'), (10, 'to'), (11, 'it'), (12, 'me'), (13, 'my'), (14, 'in'), (15, 't'), (16, 's'), (17, 'that'), (18, 'on'), (19, 'of'), (20, '.'), (21, 'your'), (22, 'm'), (23, 'we'), (24, 'all'), (25, 'like'), (26, 'is'), (27, 'be'), (28, 'for'), (29, 'up'), (30, 'so'), (31, 'with'), (32, 'can'), (33, 'love'), (34, 'know'), (35, 'just'), (36, 'but'), (37, 'don'), (38, 'no'), (39, 'oh'), (40, 'what'), (41, 'got'), (42, 'this'), (43, 'get'), (44, '?'), (45, 'they'), (46, 'when'), (47, 'do'), (48, 'she'), (49, 'yeah'), (50, 'now'), (51, 'baby'), (52, 'if'), (53, 'go'), (54, 're'), (55, 'he'), (56, 'out'), (57, 'was'), (58, 'one'), (59, '!'), (60, 'down'), (61, 'll'), (62, 'want'), (63, 'let'), (64, 'see'), (65, 'there'), (66, 'come'), (67, 'cause'), (68, 'say'), (69, 'her'), (70, 'not'), (71, 'at'), (72, 'make'), (73, 'time'), (74, 'from'), (75, 'back'), (76, 'have'), (77, 'are'), (7

In [14]:
tokenizer.word_counts

OrderedDict([('<start>', 175749),
             ('look', 1652),
             (',', 68536),
             ('i', 62775),
             ('was', 4794),
             ('gonna', 2355),
             ('go', 5080),
             ('easy', 404),
             ('on', 12722),
             ('you', 47031),
             ('and', 29409),
             ('not', 3621),
             ('to', 26804),
             ('hurt', 374),
             ('your', 11370),
             ('feelings', 109),
             ('<end>', 175749),
             ('but', 7482),
             ('m', 11183),
             ('only', 1772),
             ('going', 1015),
             ('get', 6517),
             ('this', 6776),
             ('one', 4661),
             ('chance', 289),
             ('something', 982),
             ('s', 15510),
             ('wrong', 662),
             ('can', 7669),
             ('feel', 2199),
             ('it', 22630),
             ('six', 197),
             ('minutes', 35),
             ('slim', 118),
             ('sha

In [15]:
print(tensor[:3, :10])

[[   4    5   57   96   53  413   18    7    8   70]
 [   2   36    5   22  124  194   10   43   42   58]
 [   4    5   32  101   11  712 2474    4 1012  866]]


In [16]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [17]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[   4    5   57   96   53  413   18    7    8   70   10  441   21 1079]
[   5   57   96   53  413   18    7    8   70   10  441   21 1079    3]


In [18]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 7000개와, 여기 포함되지 않은 0:<pad>를 포함하여 7001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [19]:
VOCAB_SIZE

12001

## 4) 평가 데이터셋 분리

tokenize() 로 Tensor로 변환한 후, 데이터 분리

In [20]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          test_size=0.2,
                                                          random_state=7
                                                            )

In [21]:
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140599, 14)
Target Train: (140599, 14)


## 5) 학습 모델 생성


In [22]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [23]:
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-2.77341111e-04, -2.06104218e-04, -9.92091373e-05, ...,
         -2.41189846e-05,  1.68035738e-04,  1.24072263e-04],
        [-3.77488352e-04, -2.39734611e-04,  1.10783636e-04, ...,
          1.73859007e-04,  9.28573500e-05, -4.13034541e-05],
        [-5.40067093e-04, -9.46810978e-05,  3.31036514e-04, ...,
          3.85182560e-04, -1.11576534e-04, -1.31212175e-04],
        ...,
        [ 2.37163567e-05,  1.07107204e-04,  1.63303383e-04, ...,
          1.95122935e-04,  5.66568851e-05, -5.91536460e-04],
        [ 1.03024613e-04,  4.69350380e-05,  6.97625183e-06, ...,
          2.40243637e-04,  2.24701187e-04, -5.09799458e-04],
        [-1.91440995e-05,  1.77372494e-04, -6.84893748e-05, ...,
          1.13414280e-04,  3.55536118e-04, -5.29895886e-04]],

       [[-2.77341111e-04, -2.06104218e-04, -9.92091373e-05, ...,
         -2.41189846e-05,  1.68035738e-04,  1.24072263e-04],
        [-8.35154569e-05, -2.56612839e-04, -2

In [24]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  18882560  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


In [25]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 239s 348ms/step - loss: 3.6328
Epoch 2/10
686/686 [==============================] - 240s 351ms/step - loss: 2.9877
Epoch 3/10
686/686 [==============================] - 241s 351ms/step - loss: 2.6788
Epoch 4/10
686/686 [==============================] - 240s 351ms/step - loss: 2.3931
Epoch 5/10
686/686 [==============================] - 241s 351ms/step - loss: 2.1198
Epoch 6/10
686/686 [==============================] - 241s 351ms/step - loss: 1.8598
Epoch 7/10
686/686 [==============================] - 241s 351ms/step - loss: 1.6285
Epoch 8/10
686/686 [==============================] - 241s 351ms/step - loss: 1.4350
Epoch 9/10
686/686 [==============================] - 242s 352ms/step - loss: 1.2779
Epoch 10/10
686/686 [==============================] - 244s 356ms/step - loss: 1.1560


In [26]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [27]:
generate_text(model, tokenizer, init_sentence="<start> you love ", max_len=20)

'<start> you love me when i aint sober <end> '

## Result of Project  

Seq2Seq 구조의 기본이 되는 RNN 네트워크를 학습하였고,   
자연어 처리 과정에서 중요하게 작용하는 데이터 정제 및 정규화 과정을 거치면서, **corpus | token | tokenization | Dataset 정제** 등 다양한 개념을 알게 되었고, 영어의 경우에는 그 과정에서 부호 및 줄임말을 정제해 주는 것이 중요한 것에 반해, **한국어의 경우에는 언어학적 관점에서 중요한 개념들이 많이 나왔다.** (이에 관련해서는 다양한 토큰화 라이브러리가 개발되고 있는 듯 하다.)  

처음 자연어 처리 프로젝트를 접했을 때, 네트워크 구조보다도 데이터 전처리 과정에서 나오는 용어에 대한 헷갈림이 컸는데, 이번 기회에 개념에 대한 이해는 완벽하게 잡힌 것 같다. 아직 손에 잘 익지는 않은 '텍스트'만의 특성을 지닌 과정이지만 어찌됐든 **컴퓨터가 알 수 있는 언어로 변환하는 '벡터화'과정이라는 것이 이미지 처리 과정과 공통점인 것 같다.**  

라이브러리를 사용하면 더 쉽게 접근할 수 있었지만, 직접 함수를 정의해보고(날코딩을 한 것은 아니지만) 접해보니 어떠한 과정에서 토큰화가 일어나는지 알 수 있어서 좋았다. 

## Good  

텍스트 데이터 정제 및 전처리 과정에서 활용되는 개념과 과정을 접할 수 있었다.  
활용되는 함수를 정의하여 내부 알고리즘을 경험할 수 있던 점이 도움이 됐다.

## Difficulties / Challenges  

* 토큰화를 하고 나서 Dataset 객체화 하기 전에, 토큰 15개 이상을 가진 문장을 제거하고 Dataset으로 만들어야 하는 정제 과정을 해결하지 못했다. (파이썬 문법과 그를 짜는 알고리즘이 아직도 익숙치 않은 탓이다.)  
* RNN 네트워크 구조를 더 연구하여 효율적인 레이어 구성을 시도해보고 싶다.  
* NLTK 라이브러리를 활용하여 내장 함수를 통해 토큰화하는 과정 활용 